In [1]:
import pandas as pd
import os

In [5]:
FileFrame = pd.read_csv('/home/soenke/Internship/DeepDicomSortFork/prototyping/parelsnoer/FileFrame.csv')
Results = pd.read_csv('/home/soenke/Internship/DeepDicomSortFork/prototyping/parelsnoer/Predictions_DDS_model_epochs20_time_2023-11-19_22:56_ensamblePredictions.csv')

In [6]:
FileFrame

,originPath,structuredDicomPath,NIFTI_path,sliced
0,/trinity/home/r098375/DDS/data/parelsnoer/data...,/trinity/home/r098375/DDS/data/parelsnoer/DICO...,/trinity/home/r098375/DDS/data/parelsnoer/NIFT...,True
1,/trinity/home/r098375/DDS/data/parelsnoer/data...,/trinity/home/r098375/DDS/data/parelsnoer/DICO...,/trinity/home/r098375/DDS/data/parelsnoer/NIFT...,True
2,/trinity/home/r098375/DDS/data/parelsnoer/data...,/trinity/home/r098375/DDS/data/parelsnoer/DICO...,/trinity/home/r098375/DDS/data/parelsnoer/NIFT...,True
3,/trinity/home/r098375/DDS/data/parelsnoer/data...,/trinity/home/r098375/DDS/data/parelsnoer/DICO...,/trinity/home/r098375/DDS/data/parelsnoer/NIFT...,True
4,/trinity/home/r098375/DDS/data/parelsnoer/data...,/trinity/home/r098375/DDS/data/parelsnoer/DICO...,/trinity/home/r098375/DDS/data/parelsnoer/NIFT...,True
...,...,...,...,...
8403,/trinity/home/r098375/DDS/data/parelsnoer/data...,/trinity/home/r098375/DDS/data/parelsnoer/DICO...,NaN,NaN
8404,/trinity/home/r098375/DDS/data/parelsnoer/data...,/trinity/home/r098375/DDS/data/parelsnoer/DICO...,NaN,NaN
8405,/trinity/home/r098375/DDS/data/parelsnoer/data...,/trinity/home/r098375/DDS/data/parelsnoer/DICO...,NaN,NaN
8406,/trinity/home/r098375/DDS/data/parelsnoer/data...,/trinity/home/r098375/DDS/data/parelsnoer/DICO...,NaN,NaN


In [7]:
Results

,NIFTI_name,slicenum,imageID,groundTruth,prediction,raw_0,raw_1,raw_2,raw_3,raw_4,raw_5,raw_6,raw_7,raw_8,NIFTI_path,vote,certainty
0,1.2.826.0.1.3680043.9.6827.1011249712807393890...,10,/trinity/home/r098375/DDS/data/parelsnoer/NIFT...,0,0,1.0,0.000000e+00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,/trinity/home/r098375/DDS/data/parelsnoer/NIFT...,0.0,1.000000
1,1.2.826.0.1.3680043.9.6827.1011249712807393890...,11,/trinity/home/r098375/DDS/data/parelsnoer/NIFT...,0,0,1.0,0.000000e+00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,/trinity/home/r098375/DDS/data/parelsnoer/NIFT...,0.0,1.000000
2,1.2.826.0.1.3680043.9.6827.1011249712807393890...,12,/trinity/home/r098375/DDS/data/parelsnoer/NIFT...,0,0,1.0,0.000000e+00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,/trinity/home/r098375/DDS/data/parelsnoer/NIFT...,0.0,1.000000
3,1.2.826.0.1.3680043.9.6827.1011249712807393890...,13,/trinity/home/r098375/DDS/data/parelsnoer/NIFT...,0,0,1.0,0.000000e+00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,/trinity/home/r098375/DDS/data/parelsnoer/NIFT...,0.0,1.000000
4,1.2.826.0.1.3680043.9.6827.1011249712807393890...,14,/trinity/home/r098375/DDS/data/parelsnoer/NIFT...,0,0,1.0,0.000000e+00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,/trinity/home/r098375/DDS/data/parelsnoer/NIFT...,0.0,1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11140,1.2.826.0.1.3680043.9.6827.9977247350982070909...,5,/trinity/home/r098375/DDS/data/parelsnoer/NIFT...,0,0,1.0,1.310432e-19,0.0,0.0,0.0,0.0,0.0,0.0,0.0,/trinity/home/r098375/DDS/data/parelsnoer/NIFT...,0.0,0.866667
11141,1.2.826.0.1.3680043.9.6827.9977247350982070909...,6,/trinity/home/r098375/DDS/data/parelsnoer/NIFT...,0,1,0.0,1.000000e+00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,/trinity/home/r098375/DDS/data/parelsnoer/NIFT...,0.0,0.866667
11142,1.2.826.0.1.3680043.9.6827.9977247350982070909...,7,/trinity/home/r098375/DDS/data/parelsnoer/NIFT...,0,0,1.0,0.000000e+00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,/trinity/home/r098375/DDS/data/parelsnoer/NIFT...,0.0,0.866667
11143,1.2.826.0.1.3680043.9.6827.9977247350982070909...,8,/trinity/home/r098375/DDS/data/parelsnoer/NIFT...,0,0,1.0,0.000000e+00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,/trinity/home/r098375/DDS/data/parelsnoer/NIFT...,0.0,0.866667


In [14]:
Results_depr = Results[Results['slicenum']==10][['groundTruth','vote','NIFTI_path']]

In [15]:
Results_depr

,groundTruth,vote,NIFTI_path
0,0,0.0,/trinity/home/r098375/DDS/data/parelsnoer/NIFT...
15,0,0.0,/trinity/home/r098375/DDS/data/parelsnoer/NIFT...
30,3,3.0,/trinity/home/r098375/DDS/data/parelsnoer/NIFT...
45,0,0.0,/trinity/home/r098375/DDS/data/parelsnoer/NIFT...
60,8,1.0,/trinity/home/r098375/DDS/data/parelsnoer/NIFT...
...,...,...,...
11070,0,0.0,/trinity/home/r098375/DDS/data/parelsnoer/NIFT...
11085,0,0.0,/trinity/home/r098375/DDS/data/parelsnoer/NIFT...
11100,0,0.0,/trinity/home/r098375/DDS/data/parelsnoer/NIFT...
11115,3,3.0,/trinity/home/r098375/DDS/data/parelsnoer/NIFT...


In [16]:
CombinedFrame = Results_depr.merge(FileFrame, how='inner', left_on='NIFTI_path', right_on='NIFTI_path')

In [19]:
CombinedFrame = CombinedFrame.drop('sliced', axis=1)

In [20]:
CombinedFrame

,groundTruth,vote,NIFTI_path,originPath,structuredDicomPath
0,0,0.0,/trinity/home/r098375/DDS/data/parelsnoer/NIFT...,/trinity/home/r098375/DDS/data/parelsnoer/data...,/trinity/home/r098375/DDS/data/parelsnoer/DICO...
1,0,0.0,/trinity/home/r098375/DDS/data/parelsnoer/NIFT...,/trinity/home/r098375/DDS/data/parelsnoer/data...,/trinity/home/r098375/DDS/data/parelsnoer/DICO...
2,3,3.0,/trinity/home/r098375/DDS/data/parelsnoer/NIFT...,/trinity/home/r098375/DDS/data/parelsnoer/data...,/trinity/home/r098375/DDS/data/parelsnoer/DICO...
3,0,0.0,/trinity/home/r098375/DDS/data/parelsnoer/NIFT...,/trinity/home/r098375/DDS/data/parelsnoer/data...,/trinity/home/r098375/DDS/data/parelsnoer/DICO...
4,8,1.0,/trinity/home/r098375/DDS/data/parelsnoer/NIFT...,/trinity/home/r098375/DDS/data/parelsnoer/data...,/trinity/home/r098375/DDS/data/parelsnoer/DICO...
...,...,...,...,...,...
738,0,0.0,/trinity/home/r098375/DDS/data/parelsnoer/NIFT...,/trinity/home/r098375/DDS/data/parelsnoer/data...,/trinity/home/r098375/DDS/data/parelsnoer/DICO...
739,0,0.0,/trinity/home/r098375/DDS/data/parelsnoer/NIFT...,/trinity/home/r098375/DDS/data/parelsnoer/data...,/trinity/home/r098375/DDS/data/parelsnoer/DICO...
740,0,0.0,/trinity/home/r098375/DDS/data/parelsnoer/NIFT...,/trinity/home/r098375/DDS/data/parelsnoer/data...,/trinity/home/r098375/DDS/data/parelsnoer/DICO...
741,3,3.0,/trinity/home/r098375/DDS/data/parelsnoer/NIFT...,/trinity/home/r098375/DDS/data/parelsnoer/data...,/trinity/home/r098375/DDS/data/parelsnoer/DICO...
